# Azure AI Foundry Agent Service - カスタム関数を使ったシングルエージェント

### 概要

このハンズオンでは、Azure AI Foundry Agent Serviceを使用して、カスタム関数（Function Calling）を備えたAIエージェントを作成します。

### 学習内容
- Azure AI Foundry Agent Serviceの基本的な使い方
- カスタム関数の定義と実装
- Function Callingを使ったエージェントの作成
- 外部APIとの連携（Open-Meteo天気API）
- エージェントとの対話とツール呼び出しの確認

### このサンプルで扱う機能
1. **都道府県の座標取得**: 日本の都道府県名から緯度・経度を取得
2. **現在日時の取得**: 日本標準時（JST）での現在日付を取得
3. **気温データの取得**: 指定した座標と日付の気温情報を外部APIから取得

これらの機能を組み合わせて、「東京の今日の気温を教えて」といった自然言語での質問に対して、エージェントが自動的に必要な関数を呼び出して回答を生成します。

# ライブラリのインポート

必要なPythonライブラリとAzure AI Foundry SDKをインポートします。

In [14]:
import os
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    ListSortOrder,
    FunctionTool,
    ToolSet,
)

# 環境変数の取得

Azure AI Foundryへの接続に必要な設定情報を環境変数から取得します。

In [15]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

Azure AI Foundryプロジェクトへの接続と Foundry Agent 管理用のクライアントを初期化します。

In [16]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

エージェントの実行結果を表示するためのヘルパー関数を定義します。

`agent_run_outputs`関数は以下の機能を提供します：
- スレッド内のメッセージ一覧を取得・表示
- 画像コンテンツがある場合は保存・表示

In [17]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

# ツールの定義

エージェントが使用できるカスタム関数（ツール）を定義します。

Function Calling により、エージェントは以下のような処理を自動的に実行できます：
1. ユーザーの質問を分析
2. 必要な情報を取得するための適切な関数を選択
3. 関数を呼び出して情報を取得
4. 取得した情報を基に自然な回答を生成

## カスタム関数

エージェントが利用できる3つのカスタム関数を定義します。

### 定義する関数
1. **get_prefecture_location**: 都道府県名から緯度・経度を取得
2. **get_current_time_jst**: 日本標準時の現在日付を取得
3. **get_temperature**: 指定座標・日付の気温を外部APIから取得

これらの関数は適切なdocstringとtype hintsを含むことで、エージェントが自動的に関数の用途やパラメータを理解できるようになっています。

In [18]:
def get_prefecture_location(prefecture: str) -> str:
    """
    都道府県名から緯度・経度を取得します。都道府県一覧・座標は変更しません。

    :param prefecture (str): 都道府県名（例: "東京都"）
    :rtype: str

    :return: {"latitude": ..., "longitude": ...} 形式のJSON文字列。見つからなければ {"latitude": null, "longitude": null}
    :rtype: str
    """
    # 都道府県の座標データを読み込み
    json_path ="../../infra/sample_data/prefecture_locations.json"
    with open(json_path, "r", encoding="utf-8") as f:
        PREFECTURE_LOCATIONS = json.load(f)
        
    loc = PREFECTURE_LOCATIONS.get(prefecture)
    if loc:
        return json.dumps({"latitude": loc[0], "longitude": loc[1]})
    else:
        return json.dumps({"latitude": None, "longitude": None})


def get_current_time_jst() -> str:
    """
    日本標準時（JST, UTC+9）の今日の日付を "YYYY-MM-DD" 形式の文字列で返します。日時情報は変更しません。

    :rtype: str

    :return: 今日の日付（"YYYY-MM-DD" 形式の文字列）
    :rtype: str
    """
    jst = zoneinfo.ZoneInfo("Asia/Tokyo")
    now = datetime.datetime.now(jst)
    return now.strftime("%Y-%m-%d")


def get_temperature(latitude: float, longitude: float, date: str) -> str:
    """
    指定した緯度・経度と日付に基づき、Open-Meteo APIで該当日の平均気温（摂氏）を返します。
    外部サービスのみ参照し、情報は変更しません。

    :param latitude (float): 緯度
    :param longitude (float): 経度
    :param date (str): 日付（"YYYY-MM-DD" 形式）
    :rtype: str

    :return: 指定日の平均気温（摂氏）をJSON文字列で返します。データがなければ {"temperature": null}。
    :rtype: str
    """
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        f"&start_date={date}&end_date={date}"
        "&daily=temperature_2m_mean"
        "&timezone=Asia%2FTokyo"
    )
    resp = requests.get(url, timeout=5)
    resp.raise_for_status()
    data = resp.json()
    temps = data.get("daily", {}).get("temperature_2m_mean", [])
    value = temps[0] if temps else None
    return json.dumps({"temperature": value})


## 関数のテスト

定義したカスタム関数が正常に動作することを確認するテストコードです。

In [19]:
prefecture_name = "北海道"

# 都道府県名から緯度・経度を取得
loc_json = get_prefecture_location(prefecture_name)

# 緯度・経度を取得
loc = json.loads(loc_json)
lat = loc["latitude"]
lon = loc["longitude"]

# 現在の日付を取得
today = get_current_time_jst()

# 平均気温を取得
temp_json = get_temperature(lat, lon, today)
temp = json.loads(temp_json)["temperature"]

print(f"{today} の {prefecture_name} の平均気温: {temp}°C")

2025-07-29 の 北海道 の平均気温: 24.6°C


## Toolset に格納

定義したカスタム関数をエージェントが使用できる形式（ToolSet）に格納します。

- `ToolSet`: 複数のツールをまとめて管理するコンテナ
- `FunctionTool`: Python関数をエージェントツールとして定義
- `enable_auto_function_calls`: エージェントが自動的に関数を呼び出せるように設定

この設定により、エージェントは会話の文脈に応じて適切な関数を自動選択・実行できるようになります。

In [20]:
# Toolset の作成＆関数の追加
toolset = ToolSet()

functions = FunctionTool(functions={get_prefecture_location, get_current_time_jst, get_temperature})
toolset.add(functions)

# エージェントで関数を自動的に呼び出すように設定
agents_client.enable_auto_function_calls(functions)

# エージェントの作成

Foundry エージェントを作成します。

## エージェント設定の詳細
- **model**: 使用するAIモデル（GPT-4.1 等）
- **name**: エージェントの名前
- **instructions**: システム指示文（エージェントの振る舞いを定義）
- **description**: エージェントの説明（ユーザーや他のエージェントがこのエージェントの用途や機能を理解するための情報）
- **tools**: エージェントが使用できるツールセット

このエージェントは日本の地理・気象情報に特化し、自然言語での質問に対して適切な関数を呼び出して回答します。

In [21]:
custom_functions_agent = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="custom_functions_agent",
    instructions=(
        "あなたは日本国内の気温や地理情報に答えるアシスタントです。"
        "利用可能なツールを確認し、必要に応じて使用してください。"        
    ),
    description=(
        "日本の都道府県の地理情報（緯度・経度）や、日本標準時の日付、指定した場所・日付の気温データを自動的に取得し、"
        "ユーザーの質問に答えるアシスタントです。 "
    ),
    tools=toolset.definitions,
)
print(f"Created Agent. AGENT_ID: {custom_functions_agent.id}")


Created Agent. AGENT_ID: asst_quMB3kw12eMPIz2Fcg71S2Ul


# スレッドの作成

エージェントとの会話を管理するスレッドを作成します。

## スレッドの役割
- 会話の文脈を保持
- メッセージ履歴の管理
- 複数回のやり取りにおける状態管理
- エージェントとユーザー間の対話セッションを提供

作成されたスレッドIDは、この後のメッセージ追加や実行で使用されます。

In [22]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_MVCG0L4jIp19HIEHBgdWVvjc


# ユーザーメッセージの追加

エージェントに対する質問やリクエストをスレッドに追加します。

In [23]:
# メッセージの追加
user_message = "過去３日間の沖縄県の平均気温を教えて。"
# user_message = "Tokyoの今日の気温を教えて。"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_Bu6iouwzKndijpTocl9J9R5M


# Run の実行

エージェントを実行してユーザーメッセージに対する回答を生成します。

## Runの処理フロー
1. スレッド内のメッセージを分析
2. 必要な情報を取得するためのツール選択
3. カスタム関数の自動実行
4. 取得した情報を統合して回答生成
5. スレッドメッセージへ書き込み

`create_and_process`により、エージェントの実行から完了まで自動的に処理されます。
エラーが発生した場合は、詳細なエラー情報が表示されます。

In [24]:
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=custom_functions_agent.id
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: 過去３日間の沖縄県の平均気温を教えて。
ASSISTANT: 過去3日間（7月26日～28日）の沖縄県の平均気温は以下の通りです。

- 7月28日：28.0℃
- 7月27日：28.2℃
- 7月26日：28.5℃

他にも知りたい日や地域があればお知らせください。


# トレース確認

[Azure AI Foundry Portal](https://ai.azure.com/?cid=learnDocs) でエージェントの実行トレースを確認してみましょう。

[ エージェント ] > [ 自分のスレッド ] を押下し、`最新のスレッド ID` を選択します。
画面右のスレッドプロパティから `プレイグラウンドで試す` を押下

![alt text](./../../docs/img/image-01-01.png)

エージェントプレイグラウンドが開いたら、 `スレッドログ` を押下すると以下のエージェントトレースを確認できまｓ。

![alt text](./../../docs/img/image-01-02.png)


## トレースで確認できる情報
- スレッド上の Run および Run Step のプロセス
- どのツールが選択・実行されたか
- 各関数の入力・出力パラメータ
- 実行時間とパフォーマンス情報
- Evaluations （Build-in 評価指標）
- エラーが発生した場合の詳細情報

トレース機能により、エージェントがどのように判断・処理を行ったかを詳細に分析できます。

# Agent ID を .env ファイルに保存

作成したエージェントのIDを .env ファイルに保存し、後続のハンズオン演習で再利用できるようにします。
保存された Agent ID は、マルチエージェント構成や他の機能との組み合わせで使用されます。

In [25]:
# 変数の定義
agent_env_key = "FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID"
agent_env_value = custom_functions_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv() 
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')

.envファイルに FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID="asst_quMB3kw12eMPIz2Fcg71S2Ul" を追記しました。
